In [1]:
from z3 import *

In [2]:
s = Optimize()
x = Int("x")
s.add_soft(x == 1, weight = 1)
s.add_soft(x == 2, weight = 2)
model = s.check()
print(s.model())

[x = 2]


In [3]:
s = Optimize()
x, y = Ints("x y")
s.add(x <= 10)
s.add(x >= 0)
s.add(y > 20)
s.add(y <= 30)
s.maximize(y+2*x)
s.add_soft(x < 5, weight = 100)
model = s.check()
print(s.model())

[y = 30, x = 10]


In [17]:
s = Optimize()
x, y = Ints("x y")
s.add_soft(x == 1, weight = 1)
s.add_soft(y == 2, weight = 1)
s.add_soft(x + y == 3, weight = -1)
model = s.check()
print(s.model())

[y = 2, x = 0]


In [23]:
def adjacent(x, y):
    if not (isinstance(x, z3.z3.ArithRef) and isinstance(y, z3.z3.ArithRef) and x.is_int() and y.is_int()):
        raise TypeError("Both inputs must be z3 integer variables")

    return Or(x - y == 1, x - y == -1)

def tripleChange(x, y, z):
    if not (isinstance(x, z3.z3.ArithRef) and isinstance(y, z3.z3.ArithRef) and isinstance(z, z3.z3.ArithRef) 
            and x.is_int() and y.is_int() and z.is_int()):
        raise TypeError("Both inputs must be z3 integer variables")

    return Or(And(adjacent(x, y), adjacent(y, z)), And(adjacent(y, x), adjacent(x, z)), And(adjacent(y, z), adjacent(z, x)))

TODO:
* guarantee blocks are similarly sized
* vignettes in order
* at most one vignette per block
* at most one diddy per block
* control first/last overall sketches

In [24]:
type("hello")

str

In [26]:
class Actor:
    def __init__(self, name: str):
        if not (isinstance(name, str)):
            raise TypeError("Actor name must be a string")
        self.name = name

alice = Actor("Alice")
print(alice.name)
bob = Actor(6)
print(bob.name)

Alice


TypeError: Actor name must be a string

In [86]:
class Sketch:
    def __init__(self, name, actors):
        if not (isinstance(name, str)):
            raise TypeError("Sketch name must be a string")
        seen = set([])
        if not (isinstance(actors, list)):
            raise TypeError("The second argument must be a list of actors in the sketch")
        for actor in actors:
            if not (isinstance(actor, Actor)):
                raise TypeError("Every element in list of actors must be of type Actor")
            if actor.name in seen:
                raise ValueError("Every actor in provided list must have a unique name")
            seen.add(actor.name)
            
        self.name = name
        self.actors = actors

class Diddy(Sketch):
    def __init__(self, name, actors):
        super().__init__(name, actors)

class Vignettes(Sketch):
    def __init__(self, name, actors):
        if not (isinstance(name, str)):
            raise TypeError("Name of vignettes must be a string")
        if not (isinstance(actors, list)):
            raise TypeError("The second argument must be a list where each element is a list of actors in a vignette")
        if len(actors) == 0:
            raise ValueError("Please provide the actors in at least 1 vignette")
        for vignette_actors in actors:
            if not (isinstance(vignette_actors, list)):
                raise TypeError("Every element in the second argument should be a list of actors")
            seen = set([])
            for actor in vignette_actors:
                if not (isinstance(actor, Actor)):
                    raise TypeError("Every element in list of actors must be of type Actor")
                if actor.name in seen:
                    raise ValueError("Every actor in list for given vignette must have a unique name")
                seen.add(actor.name)

        self.name = name
        self.actors = actors

In [122]:
def blockSizing(blocks, vignettes_and_diddies, n_total):
    if not (isinstance(n_total, int)):
        raise TypeError("Last argument must be an integer representing number of sketches (each vignette counted as separate sketch) plus desired number of stage blocks minus 1")
    if n_total < len(blocks) + len(vignettes_and_diddies):
        raise ValueError("Last argument must be an integer representing number of sketches (each vignette counted as separate sketch) plus desired number of stage blocks minus 1")
    if not(isinstance(blocks, list)):
        raise TypeError("First argument must be list of z3 int variables corresponding to positions between stage blocks")
    for block in blocks:
        if not(isinstance(block, z3.z3.ArithRef) and block.is_int()):
            raise TypeError("Every element of first argument must be a z3 int variable")
    if not(isinstance(vignettes_and_diddies, list)):
        raise TypeError("Second argument must be list of z3 int variables corresponding to positions of vignettes and diddies")
    for sketch in vignettes_and_diddies:
        if not (isinstance(sketch, z3.z3.ArithRef) and sketch.is_int()):
            raise TypeError("Every element of second argument must be a z3 int variable")

    blocks.insert(0, 0) 
    blocks.append(n_total + 1) #adjust list of block positions so length of any stage block will be the difference between 2 list elems

    #compute length of each block
    block_lengths = []
    for block_num in range(1, len(blocks)):
        start_pos = blocks[block_num - 1]
        end_pos = blocks[block_num]
        length = end_pos - start_pos - 1
        for short_sketch in vignettes_and_diddies:
            length = length - 0.5*And(short_sketch > start_pos, short_sketch < end_pos) #vignettes/diddies count as half a sketch for length purposes
        block_lengths.append(length)

    #ensure lengths are within 1 of 1 another
    length_conditions = []
    for i in range(len(block_lengths)):
        for j in range(i + 1, len(block_lengths)):
            length_conditions.append(block_lengths[i] - block_lengths[j] <= 1)
            length_conditions.append(block_lengths[i] - block_lengths[j] >= -1)

    return And(length_conditions)

In [142]:
sketches = []
for i in range(11):
    sketches.append(Int("s" + str(i)))

blocks = Ints("b1 b2 b3")
s = Solver()
all_vars = sketches.copy()
all_vars.extend(blocks)
for i in range(len(all_vars)):
    s.add(all_vars[i] >= 1)
    s.add(all_vars[i] <= 14)
    for j in range(i + 1, len(all_vars)):
        s.add(all_vars[i] != all_vars[j])

s.add(b2 <= 14)
s.add(blockSizing(blocks, sketches[:3], 14))
s.check()
print(s.model())

[b1 = 4,
 b3 = 12,
 s7 = 6,
 s5 = 11,
 s10 = 3,
 s9 = 7,
 s3 = 13,
 s6 = 10,
 b2 = 9,
 s2 = 5,
 s4 = 14,
 s1 = 2,
 s0 = 8,
 s8 = 1]


In [ ]:
def set_subtract(X, Y):
    if not(isinstance(X, list) and isinstance(Y, list):
        raise TypeError("Both inputs must be lists")
    for y in Y:
        if not (y in X):
            raise ValueError("Second argument must be completely contained within first")

    return_list = []
    for x in X:
        if not (x in Y):
            return_list.append(x)

    return return_list
    

#given list of elements, return all k-element subsets
def choose_k(elems, k):
    if not (isinstance(k, int)):
        raise TypeError("Second argument must be an integer")
    if k < 0 or k > len(elems):
        raise ValueError("Second must be a positive integer and at most the length of the first input")
    if not (isinstance(elems, list)):
        raise TypeError("First argument must be a list")
    subsets = []
    for _ in range(k + 1):
        new_subsets = []
        for subset in subsets:
            for new_elem in set_subtract(elems, subset):
                new_subset = subset.copy()
                new_subset.append(new_elem)
                new_subsets.append(new_subset)
        

In [6]:
import itertools
for i in itertools.combinations([5, 2, 4, 3,1], 3):
    print(i)

(5, 2, 4)
(5, 2, 3)
(5, 2, 1)
(5, 4, 3)
(5, 4, 1)
(5, 3, 1)
(2, 4, 3)
(2, 4, 1)
(2, 3, 1)
(4, 3, 1)


In [7]:
True + True

2

In [13]:
a, b, c = Bools("a b c")
s = Optimize()
s.add(Sum([a, b, c]) <= 2)
s.add_soft(a)
s.add_soft(b)
s.add_soft(c, weight = 2)
s.check()
print(s.model())

[a = False, b = True, c = True]


In [7]:
lotsOfBools = Bools("a b c d e f g")
s = Solver()
for i in range(len(lotsOfBools)):
    s.add(Implies(lotsOfBools[i], Not(Or(Or(lotsOfBools[:i]), Or(lotsOfBools[i+1:])))))
s.add(lotsOfBools[0])
s.check()
print(s.model())

[f = False,
 b = False,
 a = True,
 c = False,
 d = False,
 e = False,
 g = False]
